Skrevet av: Lasse Matias Dragsjø, Bendik Kvamme Stokland, og Thomas Olaussen

# Damptrykkurven
Damptrykkurven er et termofysisk begrep som beskriver overgangen fra vann til damp ved ulike trykk, temperatur, og volum. Den illustreres vanligvis i en plott med trykk som funksjon av temperatur med volum konstant, eller trykk som funksjon av volum med temperatur konstant.

Denne kurven, og flere slike kurver, er nyttige i termodynamikken, siden de kan anvendes til mange ulike ingeniørformål og forsøk.
Kurvene kan bli hentet på flere måter: Enten kan de løses analytisk, eller man kan bruke van der Waals ligning til å få et forenklet modell av den.

Verdiene til deres variabler kan deretter bli funnet ved å gjøre eksperimentelle forsøk.

I denne rapporten går vi gjennom hvordan vi gjør de numeriske metodene:
Vi går gjennom van der Waals tilstandsligning og løser denne og så bruker vi ulike numeriske metoder for å anslå trykket som funksjon av temperatur.

Som en oppvarming, starter vi med å se på på noen eksperimentelle verdier for van der Waals tilstandsligning.

In [ ]:
# Importerer biblioteker
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.optimize import curve_fit
from scipy import interpolate

# Gjør figurer manuelt større
plt.rcParams['figure.figsize'] = [12, 8]
# Gjør tekst større
plt.rcParams.update({'font.size': 12})

# Oppgave 1a

Vi ser på de kritiske verdiene til temperatur, trykk, og volum for van der Waals tilstandsligning. Det kritiske punktet er det punktet på kurvediagrammene der forskjellene mellom væske og gass opphører.

Van der waals tilstandsligning er gitt ved:

$p = \frac{RT}{V - b}  - \frac{a}{V^2}$,

der a og b er konstanter som man finner eksperimentelt.

De kritiske verdiene for p, T, og V er:

$T_c = 8 * \frac{a}{27R * b}$ 

$p_c = \frac{a}{27 * b^2}$

$V_c = 3b$

Ved å løse for $a$ og $b$ kan vi finne verdier for de ved eksperimentelle målinger av $T_c$ og $p_c$. Videre drøfter vi hva dette forteller oss.

In [ ]:
# Universelle gasskonstant
R = 8.31446261815324

# Eksperiementelle verdier for vann i SI-enheter
# Tatt fra oppgave teksten
T_c = 647.096 #Kelvin
p_c = 22.064000 #MegaPascal
V_c = 55.948 #MilliLiter

#fra t_c og p_c, får vi:
a = 27 * R**2 * T_c**2 / (64 * p_c)
b = R * T_c / (8 * p_c)

print(a)
print(b)

print("Volum fra b: " +  str(3 * b))
print("Volum eksperimentellt: " +  str(V_c))

Her ser vi at den eksperimentelle verdien for det kritiske volumet er $V_c \approx 55.9mL$, mens den van der Wallske gir oss $V_c \approx 91.4 mL$ når vi bruker utregnet $a$ og $b$.

Dette viser oss at selv om vi har fått $a$ og $b$ fra eksperimentelle verdier for $T_c$ og $p_c$, så er det ingen kombinasjoner av $a$ og $b$ som gir samme verdi for den eksperimentelle $V_c$.

# Oppgave 1b

Nå plotter vi van der Waals kurve med en satt $T_c$, i rekkevidden $[75ml,300ml]$, og sammenligner det med gitt kurve fra figur 2 i oppgaveteksten.

In [ ]:
# Plotteverdier
plottstart = 75
plottslutt = 300
plottmengde = (plottslutt - plottstart) + 1

# Lager plottepunktene for volum
Vplot = np.linspace(plottstart, plottslutt, plottmengde)

# Definerer van der Waals ligning
def vanderwaalP(T, V, a, b):
    
    """
    Van der Waals ligning
    
    ...
    
    Input:
    R: universelle gasskonstant
    T: temperatur
    V, volum
    a, konstant a
    b, konstant b
    
    Output:
    vanP, van der Waal trykk
    """
    
    vanP = R*T/(V-b) - a/(V**2)
    return vanP

# itererer gjennom van der Waals ligning med volumene
vanP = np.zeros(plottmengde)
for i in range(plottmengde):
    vanP[i] = vanderwaalP(T_c, Vplot[i], 27 * R**2 * T_c**2 / (64 *  p_c), R * T_c / (8 * p_c))

# Plotter    
plt.plot(Vplot, vanP)
plt.grid()
plt.xlabel("Volum [mL]")
plt.ylabel("Trykk [MPa]")
plt.title(r"Trykk kurven for vann gitt ved volum ved, $T = 647.096 K$")
plt.show()

Som figur 2 er dette en strengt synkende graf av en isoterm, men den er ikke lik den i figur 2. Vi noterer oss at den har et sadelpunkt der figur 2 har et bunnpunkt. Figur 2 stiger etter dette ekstremalpunktet til et toppunkt rundt $\approx 150mL$, men vår graf bare synker her. Kvalitativt er sadelpunktet forventet og er omtrent på riktig plass, fordi for alle $T < T_C$ vil ha et bunnpunkt der istedet. I oppgave 1g vises dette for en temperatur.

# Oppgave 1c
Vi har fått innsikt i van der Waals ligning, og går derfor over til et verktøy vi skal bruke til å numerisk utregne damptrykkurven: Newtons metode.

Vi ser på et eksempel: faseovergangen i en 2d ising-modell.
Fra uordnet til ordnet tilstand, har den kritiske temperatur denne ligningen:

$\sinh{(\frac{2c}{T_c})} = 1.$

Den analytiske løsningen til denne er:

$T_c = \frac{2c}{ln(1 + \sqrt{2})}$.

Vi bruker $c = 1K$, og finner løsningen numerisk ved hjelp av Newtons metode.

In [ ]:
# Definerer ising-modell funksjonen
def eksfuncTC(t_c):
    
    """
    ising-modell funksjonen, slik at riktig svar gir 0
    
    ...
    
    Input:
    t_c: kritisk temperatur
    
    Output:
    ising-modell funksjonen, slik at riktig svar gir 0
    """
    
    return np.sinh(2*c/t_c) - 1

# Definerer numerisk derivajson
def deriverte(func, x, h = 10e-6):
    
    """
    Numerisk derivasjon
    
    ...
    
    Input:
    func: funksjon
    x: x-punkt
    h: steglengde
    
    Output:
    deriv: f'(x), den deriverte i det punktet.
    """
    
    deriv = (func(x + h) - func(x)) / h
    return deriv

# Definerer newtons metode for én variabel.
def newtmet(func, x, tol, k):
    
    """
    Rask versjon av newtons metode for én variabel
    
    ...
    
    Input:
    func: funksjon
    x: x-punkt
    tol = feiltoleranse
    k = antall maksimum iterasjoner
    
    Output:
    x: x-punktet som er innenfor toleransebetingelsen til riktig svar
    xliste: en liste av alle x-punktene funksjonen har gått gjennom
    """
        
    xliste = np.zeros(k)
    xliste[0] = x
    for i in range(k):
        x -= (func(x) / deriverte(func, x))
        xliste[i+1] = x
        if func(x) < tol:
            break
    return x, xliste



# Henter verdier
c = 1
tol = 0.0001
k = 10000

start = 0.5
t_c = start


# Finner numerisk og analytiske svar, og printer dem
numericalTC, xlisteNewton = newtmet(eksfuncTC, t_c, tol, k)
print(numericalTC)

analyticTC = 2/np.log(1 + 2**0.5)
print(analyticTC)


# Finner numeriske svar med ulike startverdier
numericalTCliste = np.zeros(100)
for i in range(100):
    start = i
    numericalTCliste[i] = newtmet(eksfuncTC, t_c, tol, k)[0]
print(numericalTCliste)

Vi noterer oss at verdiene til den analytiske og den numeriske er veldig nærme hverandre. Vi kan derfor konkludere med at vår numeriske løsningen gir riktig svar.

Videre ser vi at verdiene $c \in [c \mid c \in ℕ \wedge c \geq 0 \wedge c \leq 100]K$ gir samme svar. Vi kan derfor konkludere med at det ikke er av betydning hvor man starter i dette inntervallet. 

# Oppgave 1d

Før vi bruker vår numeriske Newtons metode, burde vi konkludere numerisk med at den konvergerer ved ordentlige startpunkt.

Vi finner ut av dette numerisk, ved å bruke en formel som viser omtrentlig konvergensraten $q$. Vi vil forventa at Newtons metode vil ha en verdi $q = 2$, altså den konvergerer kvadratisk.

Først finner vi $e_i = |x_i - r|$, der r er roten til funksjonen vi ser på. Kvalitativt ser vi hvor langt unna roten vi er.

Så regner vi ut følgen $q_i$ som burde konvergere mot 2 for Newtons metode. Følgen er definert;

$p_i = log(\frac{e_i}{e_{i-1}}) / log(\frac{e_{i-1}}{e_{i-2}})$

In [ ]:
#  Utregner e_i
e_i = abs(xlisteNewton - analyticTC)

# Fjærner unødvendige verdier for plotting, funksjonen blir konstant
plotting_e_i = np.zeros(100)
for val in range(100):
    plotting_e_i[val] = e_i[val]

# Plotting
plt.plot(plotting_e_i)
plt.plot()
plt.grid()
plt.xlabel("Verdi for i")
plt.ylabel(r"Verdi  for $e_i$")
plt.title(r"Kurven til $e_i$ ved økende i")
plt.show()

# Utregning av p_i
p_i = np.zeros(9)

# Forsiktig med rangen her, blir fort uendelig store tall!
for xx in range(2,9):
    p_i[xx] = np.log(e_i[xx] / (e_i[xx - 1])) / np.log(e_i[xx - 1]/e_i[xx - 2])


# Fjærning av de to nullene foran 
index = [0,1]
# Bare for å være ordentlig forsiktig at det ikke blir tull
p_i = np.copy(np.delete(p_i, index))

print("Utregnet q: " + str(np.mean(p_i)))

Grafen for $e_i$ viser oss at vi kommer raskt veldig nært $r$, men så, ved $i \approx 15$ kommer vi i en 'loop' med høyere $e_i$. Uansett hvor mye vi øker $i$ vil vi få samme $e_i$.

Vi ser at verdien for $q$ er $\approx2$, men ikke lik 2, som nevnt er verdien for Newtons metode.

Vi får en verdi som er litt lavere enn den analytiske, men det er trolig grunnet numeriske feil. Noter at i utregningen har vi kun med noen få verdier av $p_i$ for å regne gjennomsnittet. Dette er på grunn av at tallene blir så store/små at vi ikke ville fått noen vetig beregning av $q$. 

# Oppgave 1e
Nå som vi har verifisert konvergens, kan vi utvide Newtons metode til flere variabler. Da kan vi regne ut nullpunktet til et sett med ligninger. Dette vil vi bruke til å løse et sett med ligninger numerisk og se hva som skjer med volumet. De to ligningene kommer fra van der Waal tilstandsligning og er:

$ p_0 = \frac{RT}{V_g - b} - \frac{a}{V_g^2} = \frac{RT}{V_v - b} - \frac{a}{V_v^2} $. 

$ \frac{RT}{V_g - V_v} * ln(\frac{V_g - b}{V_v - b}) - \frac{a}{V_g * V_v} = \frac{RT}{V_g - b} - \frac{a}{V_g^2} $

Der $V_v$ er væskevolumet og $V_g$ er gassvolumet.


Vi løser dem med hensyn på $V_v$ og $V_g$ som funksjon av temperatur på intervallet $[274,647]K$, og kommenterer resultatene. Vi vil bruke startbetingelsene gitt i oppgaveteksten for å løse de.

In [ ]:
# Newtons metode på to variabler
def NewtonTwoVariable(F, J, X, max_i, tol = 10e-4):
    
    """
    Løser et ikke lineært system med newtons to variabel metode.
    
    Input:
    F: Funksjonene slik at F = 0\n
    J: Jacobi matrisen til F\n
    X: Start verdier\n
    max_k: max antall iterasjoner\n
    tol: Toleranse
    
    Output:
    X: verdiene til nullpunkt
    """
    
    F_val = F(X)
    F_norm = np.linalg.norm(F_val)
    iter = 0
    while(abs(F_norm) < tol and iter <= max_i):
        delta = np.linalg.solve(J(X), - F_val)
        X = X + delta
        F_val = F(X)
        F_norm = np.linalg.norm(F_val)
        iter += 1
    return X

#Definerer funksjonene (11) og (12) fra guiden vår
def func(V,T,a,b):
    
    """
    Likningssystemet som skal løses
    
    Input:
    V: Liste med variabler (Volum), V[0] = V_v, V[1] = V_g
    T: Temperatur
    a: Konstant
    b: Konstant
    
    Output:
    V_list: Utregnet svar
    """
    
    #Lager en liste for systemet av likninger
    V_list=np.zeros(2)
    
    #Regner ut
    V_list[0]=(R*T)/(V[1]-b)-a/(V[1]**2)-(R*T)/(V[0]-b)+a/(V[0]**2)
    V_list[1]=R*T/(V[1]-V[0])*np.log((V[1]-b)/(V[0]-b))-a/(V[1]*V[0])-R*T/(V[1]-b)+a/(V[1]**2)
    return V_list

#Funksjon som regner jacobimatrisen til func
def Jacobi(V,T,a,b):
    
    """
    Jacobi matrisen til likningssystemet func
    
    Input:
    V: Liste med variabler (Volum), V[0] = V_v, V[1] =V_g
    T: Temperatur
    a: Konstant
    b: Konstant
    
    Output:
    matrix: Jacobi matrisen til likningsystemet func
    """
    
    #Oppretter Jacobi matrise   
    matrix = np.zeros((2,2))
    
    #Regner ut Jacobi matrisen
    matrix[0,0] = R*T/((V[0]-b)**2)-2*a/(V[0]**3)
    matrix[0,1] = -R*T/((V[1]-b)**2)+2*a/(V[1]**3)
    matrix[1,0] = -R*T/((V[1]-V[0])*(V[0]-b))+V[1]*a/((V[1]*V[0])**2)+R*T*np.log((V[1]-b)/(V[0]-b))/((V[1]-V[0])**2)
    matrix[1,1] = R*T/((V[1]-V[0])*(V[1]-b))+R*T/((V[1]-b)**2)+V[0]*a/((V[1]*V[0])**2)-2*a/(V[1]**3)-R*T*np.log((V[1]-b)/(V[0]-b))/((V[1]-V[0])**2)
    return matrix

# Newtons metode for flere funksjoner
def newtonMultiple(func, Jacobi, x, T, a, b, h=0.0001, tol=0.0001, k=1000):
    
    """
    Funksjonen bruker Newtons metode for å løse likningssytemet
    
    Input:
    func: Likningssystemet som skal løses
    Jacobi: Jacobi matrisen til likningssystemt
    x: Startsverdi for variablene som det skal løses for
    T: Temperatur
    a: Konstant
    b: Konstant
    h: Steglengde
    tol: Toleranse
    k: Maks antall iterasjoner
    
    Output:
    x_list[-1]: Nullpunkt for likningsystemet
    """
    
    #Oppretter liste av lister for å lagre nullpunkter og setter inn startsverdi
    x_list=np.zeros((k+1,len(x)))
    x_list[0]=x
    
    #Newtons metode
    for i in range(k):
        x_list[i+1] = x_list[i]-np.linalg.inv(Jacobi(x_list[i],T,a,b))@func(x_list[i],T,a,b)
        if abs(func(x_list[i+1],T,a,b)).max()<tol:
            x_list = x_list[0:i+2]
            break
    
    #Returnerer det siste steget
    return x_list[-1]



#Regner ut V_v og V_g for forskjellige T
#Lager liste med T verdier mellom T_lower og T_upper
T_lower = 274
T_upper = 647
T_list = np.linspace(T_lower,T_upper,T_upper-T_lower+1)

#Lager list for å lagre V_v og V_g
V_v = np.zeros(len(T_list))
V_g = np.zeros(len(T_list))

#Setter startspunkt for newtons metode (V_0[0]: V_v, V_0[1]: V_g)
V_0 = np.array([12658,35.6])

#Regner ut de først V_v og V_g med utgangspunkt i V_0 og T_lower
V_v[0] = newtonMultiple(func,Jacobi,V_0,T_lower,a,b)[0]
V_g[0] = newtonMultiple(func,Jacobi,V_0,T_lower,a,b)[1]

#Regner ut resten av V_v og V_g med forskjellig T
V_list = np.array([V_v[0],V_g[0]])
for i in range(1,len(T_list)):
    V_list = newtonMultiple(func,Jacobi,V_list,T_list[i],a,b)
    V_v[i] = V_list[0]
    V_g[i] = V_list[1]

#Plotter
plt.plot(V_v,T_list,label=r"$V_v$")
plt.title(r"$V_v$")
plt.ylabel("Temperatur [K]")
plt.xlabel("Volum [mL]")
plt.grid()
plt.legend()
plt.show()
plt.plot(V_g,T_list,label="V_g")
plt.title(r"$V_g$")
plt.ylabel("Temperatur [K]")
plt.xlabel("Volum [mL]")
plt.grid()
plt.legend()
plt.show()
plt.plot(V_v,T_list,label=r"$V_v$")
plt.plot(V_g,T_list,label=r"$V_g$")
plt.title(r"$V_v$ og $V_g$ plottet sammen logaritmisk")
plt.ylabel("Temperatur [K]")
plt.xlabel("Volum [mL]")
plt.grid()
plt.xscale("log")
plt.legend()
plt.show()

For $V_g$, kvalitativt, ser vi at den går raskt opp, og så dale, helt til den når toppen ved kritisk temperatur.

For $V_v$, starter den høyt og daler den raskt.

Når den nermer seg det kritiske temperaturpunktet, vil volumforskjellene til $V_v$ og $V_g$ nærme et punkt som er lik for dem begge. Dette ser vi i det siste plottet.
Altså vil $\Delta V$ gå til 0 ved $T_c$. 

# Oppgave 1f

Nå vil vi bruke verdiene vi fant og Newtons flervariabel metode til å beregne damptrykket og plotte det sammen med den eksperimentelle måleserien.

Vi henter de eksperimentelle verdier fra nettsiden https://www.engineeringtoolbox.com/water-properties-d_1573.html.

In [ ]:
# 1 bar = 100'000 Pa
barToMPa = 1e-1

TempK = np.array([273.16, 280, 300, 320, 340, 360, 380, 400, 420, 440, 460, 480, 500, 530, 560, 590, 620, 630, 640, 647.1])
TrykkP = barToMPa * np.array([6.1E-03, 0.0099, 0.0354, 0.105, 0.272, 0.622, 1.29, 2.46, 4.37, 7.34, 11.7, 17.9, 26.4, 44.6, 71.1, 108, 159, 180, 203, 221])

# Verdier som passer for Vv, Vg, og L:
TrykkVerdier = barToMPa * np.array([6.1E-03, 0.0354, 0.622, 26.4, 203, 221])

#Regner ut trykkene fra V_v og V_g med ulike T
P_listv = np.zeros(len(T_list))
P_listg = np.zeros(len(T_list))
for i in range(len(T_list)):
    P_listv[i] = vanderwaalP(T_list[i], V_v[i], a, b)
    P_listg[i] = vanderwaalP(T_list[i], V_g[i], a, b)

    
    

#Plotter de eksperimentelle og koeksistensverdiene sammen
plt.plot(TempK,TrykkP,label=r"$Eksperimentelle$")
plt.plot(T_list,P_listv,label="Koeksistensielle (v)")
plt.plot(T_list,P_listg,label="Koeksistensielle (g)")
plt.title(r"Eksperimentelle og koeksistensielle verdier plottet sammen logaritmisk")
plt.grid()
plt.yscale("log")
plt.ylabel("Trykk [MPa]")
plt.xlabel("Temperatur [K]")
plt.legend()
plt.show()

De numeriske koeksistensielle verdiene er høyere enn den eksperimentelle ved lave temperaturer, de blir mer nøyaktige dess høyere temperaturen blir. 

Dette viser oss at den 'ekte' kurven er mer sensitiv for lave temperaturer.

# Oppgave 1g
Nå vil vi se hva som skjer om vi isteden varierer volumet og setter temperaturen fast. Vi vil også se hvor punktene $(V_v, p(V_v))$ og $(V_g,p(V_g))$ og diskutere hva som fysisk skjer på intervallet $[V_v, V_g]$.


In [ ]:
#Vi velger temperatur = 550K, med tilhørende verdier. v på enden er bare å få dem til et annet variabelnavn.
Tv = T_list[550-T_lower]
Vvv = V_v[550-T_lower]
Vgv = V_g[550-T_lower]
Pvv = P_listv[550-T_lower]
Pgv = P_listg[550-T_lower]

# Volumrom, og regner ut P(V)
volume_space = np.linspace(45,10000,10046)
V_listforplot = np.zeros(len(volume_space))
for i in range(len(volume_space)):
    V_listforplot[i] = vanderwaalP(Tv, volume_space[i], a, b)
    
# Plot
plt.plot(volume_space, V_listforplot, label="P(V)")
plt.plot(Vvv, Pvv, 'bo', label=r"Punkt $(V_v,p(V_v))$")
plt.plot(Vgv, Pgv, 'ro', label=r"Punkt $(V_g,p(V_g))$")
plt.axline((Vvv,Pvv),(Vgv,Pgv), ls="--", c="r",label=r"Linje Gjennom $(V_v,p(V_v)$ og $(V_v,p(V_g)$))")
plt.xlabel("Volum [mL]")
plt.ylabel("Trykk [MPa]")
plt.title("Trykk som funksjon av volum ved T = 550K")
plt.xlim(0,350)
plt.legend()
plt.show()

Kvalitativt ser det ut som et riktig van der Waal plott. Dette kan begrunnes ved at vi har et bunnpunkt og ikke et sadelpunkt, fordi $T < T_c$.

Fysisk er van er Waal plottet feil mellom $(V_v, p(V_v))$ og $(V_g,p(V_g))$. Dette er fordi vannet opplever en faseovergang fra væske til gass (fordamping) eller omvendt (kondensasjon) der. Faseovergangen skjer under konstant trykk, som visualiseres med den rette linjen.

# Oppgave 1h

Ved å bruke resultatene så langt vil vi lage et p-V fasediagram for vann.

In [ ]:
scalefix = 1/50

plt.plot(V_v,T_list * scalefix,label=r"$V_v$")
plt.plot(V_g,T_list * scalefix,label="V_g")
plt.plot(volume_space, V_listforplot, label="P(V)")
plt.plot(Vvv, Pvv, 'bo', label=r"Punkt $(V_v,p(V_v))$")
plt.plot(Vgv, Pgv, 'ro', label=r"Punkt $(V_g,p(V_g))$")
plt.axline((Vvv,Pvv),(Vgv,Pgv), ls="--", c="r",label=r"Linje Gjennom $(V_v,p(V_v)$ og $(V_v,p(V_g)$))")
plt.text(60, 30,'Fase 1',size="large")
plt.text(95, 8,'Fase 2',size="large")
plt.text(580, 7,'Fase 3',size="large")
plt.title(r"$V_v og V_g$ plottet sammen,")
plt.grid()
plt.xscale("log")
plt.ylabel("Trykk [MPa]")
plt.xlabel("Volum [mL]")
plt.legend()
plt.show()

Vi har tre faser i dette p-V diagrammet.

Fase 1 er vann i væske fasen sin. Denne fasen er den grønne linjen ned til $(V_g,p(V_g))$.

Fase 2 er vann i et tofase område. Vann opplever en faseovergang fra væske til gass. Dette område er fra og med $(V_g, p(V_g))$ til og med $(V_v,p(V_v))$. Her er van der Waal sin løsning ufysisk og blir utvekslet med den rette linjen mellom disse punktene. Dette er fordi faseovergangen skjer ved konstant trykk.

Til slutt er det fase 3, som er $(V_g, p(V_g))$ og utover. I denne fasen er vann i gassform.

# Oppgave 2a

Nå som vi har vist at van der Waals er en god tilnærming, men er ikke helt tilstrekkelig til å beskrive fasediagrammet i sin helhet, går vi over til eksperimentelle verdier. Vi vil prøve ulike metoder for å muligens forbedre diagrammene.

Vi henter inn informasjon fra samme nettside som vi gjorde i 1f.

In [ ]:
# L: https://www.engineeringtoolbox.com/water-properties-d_1573.html
# De andre: https://www.engineeringtoolbox.com/water-properties-temperature-equilibrium-pressure-d_2099.html

# Vi har 1 mol
Mm = 18E-3 # kg/mol
m3tomL = 1000000

# Disse verdiene er hentet for å matche verdiene med L2 og andre verdier så nerme som mulig.
#               0.01C 26.9C 86.9C 227C 367C 373.946C 
T2 =  np.array([273.16, 300, 360, 500, 640, 647.1])
Vg2 = np.array([Mm/0.00485, Mm/0.02558, Mm/0.3786, Mm/13.20, Mm/177.1, Mm/322.0]) 
Vv2 = np.array([Mm/999.8, Mm/996.5, Mm/967.4, Mm/831.3, Mm/481.5, Mm/322.0])
#                0.01C  25C     90C   220C  360C  373.95C
L2  = np.array([45054, 43988, 41120, 33462, 12967, 0]) # Joule / mol 

Vær merksom på at den latente varmen ($L$) ble tatt fra en annen tabell, med en viss forskjell i temperatur. Temperaturen til den latente varme målingen vises i en kommentar over de spesifikke verdiene. Dette vil utgjøre en feilkilde.

# Oppgave 2b

Første metode vi bruker for å få glatte funksjoner fra diskrete punker er kurve interpolering. Ideen er at vi definerer en funksjon med ukjente parametere som vi tror kan beskrive punktene bra. Så finner vi de beste parameterene. Her må man ofte først gjøre en god første gjetning og prøve seg fram.

For å finne parameterene brukes scipy.optimize.curve_fit for å få $V_v(t)$, $V_g(t)$ og $L(t)$.

In [ ]:
def Vv2t_func(X,a,b,c,d,e):
    
    """
    funksjon for V_v
    
    Input:
    X: variabelinout
    a til e: konstanter for funksjonen
    
    Output: En funksjon for V_v som passer godt
    """
    
    return  a*np.exp(X)+b -c*X**2 + d*X + e*X**3

def Vg2t_func(X,a,b,c,):
    
    """
    funksjon for V_g
    
    Input:
    X: variabelinout
    a til c: konstanter for funksjonen
    
    Output: En funksjon for V_v som passer godt
    """
    
    return  a*(b**X) + c

#Plotter for funksjonene for V_v og V_g
fig_t, (ax_t_vg, ax_t_vv, ax_t_l) = plt.subplots(1,3)

popt_vv2, covt_vv2 = curve_fit(Vv2t_func,T2,Vv2)
popt_vg2, covt_vg2 = curve_fit(Vg2t_func,T2,Vg2)

ax_t_vg.plot(T2,Vg2,"r+", label=r"Data for $V_g(T)$")
ax_t_vg.plot(T2, Vg2t_func(T2,*popt_vg2), c="r", label=r"Curvefit for $V_g(T)$")
ax_t_vg.set_xlabel("Temperatur [K]")
ax_t_vg.set_ylabel(r"Volum [$m^3$]") 
ax_t_vg.legend()

ax_t_vv.plot(T2,Vv2, "b+", label=r"Data for $V_v(T)$")
ax_t_vv.plot(T2, Vv2t_func(T2,*popt_vv2), c="b", label=r"Curvefit for $V_v(T)$")
ax_t_vv.set_xlabel("Temperatur [K]")
ax_t_vv.set_ylabel(r"Volum [$m^3$]") 
ax_t_vv.legend()

def l2t_func(X,a,b,c):
    
    """
    funksjon for V_g
    
    Input:
    X: variabelinout
    a til d: konstanter for funksjonen
    
    Output: En funksjon for V_v som passer godt
    """
    return a*abs(T_c - X)**b + c
    #return d*X**3 + a*X**2 + b*X + c*X**2 + e

#Plotter for funksjonen for L
popt_L2, covt_L2 = curve_fit(l2t_func,T2,L2)

ax_t_l.plot(T2,L2,"r+", label=r"Data for $L(T)$")
ax_t_l.plot(T2, l2t_func(T2,*popt_L2), c="r", label=r"Curvefit for $L(T)$")
ax_t_l.set_xlabel("Temperatur [K]")
ax_t_l.set_ylabel(r"Latent Varme [$Joule/Mol$]") 
ax_t_l.legend()
fig_t.suptitle("Kurve interpolering av innhentet data")
fig_t.tight_layout()
plt.show()

Her ser vi kurveinterpoleringen for de ulike dataene. Vi noterer oss at de har 'knekk' i de diskrete data punktene, og er derfor ikke perfekte kurver. En økning i antall punkter vil minke denne unøyaktigheten, opp til en viss grad.

Vi kan konkludere med at curvefiten passer godt med de valgte datapunktene.

# Oppgave 2c
Nå som vi har fått noe lunde rimelige funksjoner utifra de diskrete punktene vil vi numerisk utregne $p(t)$ ved hjelp av ligningen;

$dp = \frac{L(T)}{T(V_g(T) - V_v(T))}dT $

For å løse denne vil vi bruke Simpsons metode.

In [ ]:
n = abs(274-647-1)
T_intervall = np.linspace(274,647,n)

a = T_intervall[0]
b = T_intervall[-1]

h = (b-a)/n

def p_int(T):
    
    """
    Interpolasjon
    
    Input:
    T: temperatur
    
    Output: Verdi for p
    """
    
    return l2t_func(T, *popt_L2)/(T*(Vg2t_func(T,*popt_vg2) - Vv2t_func(T,*popt_vv2)))

I_simp = p_int(a)

I_simp_val = np.zeros(n)
I_simp_val[0] = p_int(a)

for i in range(1,n):
    if (i % 2 == 0):
        I_simp += 2*p_int(T_intervall[i])
        I_simp_val[i] = I_simp
    elif(i == n):
        I_simp+= p_int(T_intervall[i])
        I_simp_val[i] = I_simp
    else:
        I_simp += 4*p_int(T_intervall[i])
        I_simp_val[i] = I_simp
    

I_simp = I_simp*h/3


plt.title("Simpsons integrering av likning (13), Kurve interpolering")
plt.xlabel("Temperatur [K]")
plt.ylabel("Trykk [MPa]")
plt.plot(T_intervall,1e-6*I_simp_val)
plt.title("Trykk som funksjon av temperatur, kurve interpolering")
plt.show()
plt.plot(TempK,TrykkP)
plt.xlabel("Temperatur [K]")
plt.ylabel("Trykk [MPa]")
plt.title("Trykk som funksjon av temperatur, eksperimentelle verdier")
plt.show()

Vår numeriske metode underestimerer trykket med en faktor på $\approx 30$. Vi vet at de diskrete punktene vi brukte har litt lavere temperatur enn de andre punktene. Det er også mulig at vi ikke har brukt nok punkter. Dette kan utgjøre en del av den forskjellen vi ser.

# Oppgave 2d
Som sett var kurve interpolasjonen en god tilnærming til datapunktene. Men, gode individuelle funksjoner som passer med dataene kan være vanskelige å finne.

En metode som unngår gjetting er kubiske spliner. Den er bygd opp av mindre delspliner mellom hvert punkt. Denne spesielle oppbygningen gjør at tilsammen er den kubiske splinen et sett med kubiske polynomer. 

For å finne polynomkoeffisientene brukes scipy.interpolate.CubicSpline. Vi vil så sammenligne med den forrige metoden.

In [ ]:
interpol_vv = interpolate.CubicSpline(T2,Vv2)
interpol_vg = interpolate.CubicSpline(T2,Vg2)
interpol_l  = interpolate.CubicSpline(T2,L2)

fig_2d, (ax_t_vg2d, ax_t_vv2d, ax_t_l2d) = plt.subplots(1,3)

popt_vv2, covt_vv2 = curve_fit(Vv2t_func,T2,Vv2)
popt_vg2, covt_vg2 = curve_fit(Vg2t_func,T2,Vg2)

ax_t_vg2d.plot(T2,Vg2,"r+", label=r"Data for $V_g(T)$")
ax_t_vg2d.plot(T2, Vg2t_func(T2,*popt_vg2), c="r", label=r"Curvefit for $V_g(T)$")
ax_t_vg2d.plot(T2,interpol_vg(T2),c='g', label="Kubisk Spline")
ax_t_vg2d.set_xlabel("Temperatur [K]")
ax_t_vg2d.set_ylabel(r"Volum [$m^3$]") 
ax_t_vg2d.legend()

ax_t_vv2d.plot(T2,Vv2, "b+", label=r"Data for $V_v(T)$")
ax_t_vv2d.plot(T2, Vv2t_func(T2,*popt_vv2), c="b", label=r"Curvefit for $V_v(T)$")
ax_t_vv2d.plot(T2,interpol_vv(T2),c='g', label="Kubisk Spline")
ax_t_vv2d.set_xlabel("Temperatur [K]")
ax_t_vv2d.set_ylabel(r"Volum [$m^3$]") 
ax_t_vv2d.legend()

ax_t_l2d.plot(T2,L2,"r+", label=r"Data for $L(T)$")
ax_t_l2d.plot(T2, l2t_func(T2,*popt_L2), c="r", label=r"Curvefit for $L(T)$")
ax_t_l2d.plot(T2,interpol_l(T2),c='g', label="Kubisk Spline")
ax_t_l2d.set_xlabel("Temperatur [K]")
ax_t_l2d.set_ylabel(r"Latent Varme [$Joule/Mol$]") 
ax_t_l2d.legend()
fig_2d.suptitle("Figur av Kubisk spline mot Kurve interpolering")
fig_2d.tight_layout()
plt.show()

Her ser vi at for $V_g(T)$, $V_v(T)$ og $L(T)$ stemmer de to metodene godt overens, både med hverandre og dataene. De er tilnærmet like.
 
Merk at vi fortsatt ikke har noen kvalitativt glatt funksjon, altså funksjonene 'knekker' ved de diskrete punktene. Som sagt vil øking av antall punkter minske denne effekten.


# Oppgave 2e
Vi vil nå gjøre akkurat det samme med disse kubiske splinene som vi gjorde med kurve interpoleringen tidligere. Vi vil så diskutere de potensielle avvikene mellom grafene og hvorvidt metoden vi har brukt egentlig er gyldig i hele temperaturintervallet.

In [ ]:
def p_interpol(T):
    
    """
    Interpolasjon
    
    Input:
    T: temperatur
    
    Output: Verdi for p
    """
    
    return interpol_l(T)/T*(interpol_vg(T) - interpol_vv(T))

n = abs(274-647-1)
T_intervall = np.linspace(274,647,n)

a = T_intervall[0]
b = T_intervall[-1]

h = (b-a)/n

I_simp_inter = p_interpol(a)

I_simp_val_inter = np.zeros(n)
I_simp_val_inter[0] = p_interpol(a)

for i in range(1,n):
    if (i % 2 == 0):
        I_simp_inter += 2*p_interpol(T_intervall[i])
        I_simp_val_inter[i] = I_simp_inter
    elif(i == n):
        I_simp_inter += p_interpol(T_intervall[i])
        I_simp_val_inter[i] = I_simp_inter
    else:
        I_simp_inter += 4*p_interpol(T_intervall[i])
        I_simp_val_inter[i] = I_simp_inter
    

I_simp_inter = I_simp_inter*h/3

plt.title("Simpsons integrering av likning (13), Kubisk spline")
plt.xlabel("Temperatur [K]")
plt.ylabel("Trykk [MPa]")
plt.plot(T_intervall,1E-6*I_simp_val_inter)
plt.show()

Denne metoden, selv om den er så lik kurve interpolasjonen for $V_g(T)$, $V_v(T)$ og $L(T)$, gir helt ulik trykk. Den er ikke i samsvar med noen av de andre kurvene.

Det er flere faktorer som kan forklare avviket mellom grafene som vi har numerisk regnet ut. Den mest åpenbare er at noe er kodet feil, men det er flere ikke menneskelige faktorer som spiller inn. Det er brukt få punkter for å konstruere disse grafene i de to metodene. Øking av antall punkter vil gjøre at metodene samsvarer med det eksperimentelle mer.

Også på grunn av dette antallet punkter kan det være at interpolasjonen har så grov feil mellom punkter at den ikke er gyldig i enkelte intervall mellom de. Dette gjelder spesielt om det er stor forandring av verdien til den enkelte funksjonen. 

Øking av antall punkter brukt i Simpson integreringen vil også minske forskjellen.


# Oppgave 2f

Nå vil vi plotte alle de ulike metodene for utregning av koeeksistenskurven $p(t)$, sammen med den analytiske løsningen. $L$ blir valgt slik at den stemmer mest mulig over med eksperiementelle verdier, og referanse punktet er det kritiske punktet.

In [ ]:
# Finne et punkt (p0,t0), bruker krital punktet
p0_an = p_c
L_an = 40000 # Vi må finne egen verdi som passer best
T0_an = T_c

analytic_solution = p0_an*np.exp((L_an/R) * (1/T0_an - 1/T_intervall))

plt.plot(T_intervall, analytic_solution, label="Analytisk løsning, L = {}".format(L_an))
plt.plot(T_intervall,1e-6*I_simp_val,label="Kurve interpolering")
plt.plot(T_intervall,1e-6*I_simp_val_inter,label="Kubisk spline")
plt.plot(T_list,P_listg,label="Koeksistensielle (g)")
plt.plot(TempK,TrykkP,label=r"$Eksperimentelle$")
plt.xlabel("Temperatur [K]")
plt.ylabel("Trykk [MPa]")
plt.legend()
plt.show()

Vi ser at den kubiske splinen er helt feil. Med denne metoden er det trygt å si at den enten er kodet feil eller er ikke en god metode å numerisk regne ut $p(t)$. Resten derimot er ulik grad av akseptable.

Vi ser at den koeksistensielle fra 1f overestimerer trykket mellom de to endepunktene $[274,647]K$, men samsvarer relativt godt i endepunktene. Dette er ikke overraskende ettersom van der Waal er en god, men ikke perfekt modell av gasser og væsker.

Kurve interpoleringen starter som en god tilnærming til omtrent $T \approx 350K$. Mellom $[350,\approx 540]K$ overstimerer den trykket. Etter det undervurderer den trykket. 

Det at kurve interpoleringen var en bedre metode for å estimere $p(t)$ enn den kubiske splinen og at den kubiske splinen var så dårlig overrasket oss. Dette er fordi vi tror at en må treffe ganske godt med de funksjonene i kurve interpoleringen for å få nære resultater. Derfor regner vi med at vi med god sannsynlighet kan si at ikke er helt riktig med implementeringen av kubisk spline.
